## 영상불러와서 OD

In [4]:
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

width,height = 0,0

def detect_objects_on_video(video_path, model, threshold=0.5):
    # 비디오를 열고 비디오의 프레임을 처리합니다.
    global width,height

    cap = cv2.VideoCapture(video_path)

    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 객체 검출 실행
detect_objects_on_video('path_to_your_video.mp4', model)


d:\Anaconda3_envs\gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3_envs\gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 내장카메라

In [2]:
import cv2
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at C:\Users\circleci\project\torchvision\csrc\ops\cpu\nms_kernel.cpp:112 [kernel]
QuantizedCPU: registered at 

In [ ]:
import torch.onnx 

    

#Function to Convert to ONNX 
def Convert_ONNX(): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(1, (width,height), requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "ImageClassifier.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX')   

In [ ]:
Convert_ONNX() 